# Project - Spark SQL

Analysis of large datasets is being performed at
an unprecedented frequency. Several technologies have been
developed to do so, offering a variety of solutions and drawbacks
related to the processing of different data types and
data processing requirements. 

This notebook implements SparkSQL in order to solve a series of questions by using a data set regarding air polution in the USA.
In the report, we compared the performance
of five different technologies – MapReduce, Spark RDD,
SparkDF, Spark SQL and Hive.

In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

C:\Users\ggpt9\anaconda3\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Q.1) Which states have more/less monitors? (Rank states!)


In [12]:
try :
    lines = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./epa_hap_daily_summary-small.csv')
    lines.createOrReplaceTempView('lines')
    t1 = spark.sql("SELECT state_code, collect_set(state_name) AS StateName, count(DISTINCT latitude+longitude) AS NumberOfMonitors FROM lines GROUP BY state_code ORDER BY StateName ASC")
    #t1.createGlobalTempView("nmps")
    t1.show()
except Exception as e:
    print(e)

+----------+--------------------+----------------+
|state_code|           StateName|NumberOfMonitors|
+----------+--------------------+----------------+
|        01|           [Alabama]|              31|
|        02|            [Alaska]|              12|
|        04|           [Arizona]|              38|
|        05|          [Arkansas]|              11|
|        06|        [California]|             170|
|        08|          [Colorado]|              51|
|        09|       [Connecticut]|              15|
|        80| [Country Of Mexico]|              18|
|        10|          [Delaware]|               6|
|        11|[District Of Colu...|               5|
|        12|           [Florida]|              55|
|        13|           [Georgia]|              35|
|        15|            [Hawaii]|               5|
|        16|             [Idaho]|              17|
|        17|          [Illinois]|              49|
|        18|           [Indiana]|              52|
|        19|              [Iowa

Q.2) Which counties have the best/worst air quality? (Rank counties considering pollutants’ level!)


In [14]:
try :
    lines = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./epa_hap_daily_summary-small.csv')
    lines.createOrReplaceTempView('lines')
    t1 = spark.sql("SELECT county_name, sum(arithmetic_mean)/count(arithmetic_mean) AS Value FROM lines GROUP BY county_name ORDER BY Value DESC")
    t1.show() 
except Exception as e:
    print(e)

+---------------+------------------+
|    county_name|             Value|
+---------------+------------------+
|         Tipton|            2556.0|
|         Nassau|              19.0|
|     Columbiana| 7.385690735785953|
|CHIHUAHUA STATE|         4.5121875|
|       Caldwell| 4.116666666666666|
|         Madera|            3.7393|
|        Oakland| 2.888877848101266|
|          Duval| 2.779460397849462|
|         Kearny|2.3753333333333333|
|          Bucks|2.3674999999999997|
|San Luis Obispo|2.3333333333333335|
|      Edgecombe|             2.325|
|         Pawnee|2.2941176470588234|
|    Westchester|          2.239375|
|       Johnston|             2.225|
|       Hartford|2.0787055896226416|
|      Granville|2.0285714285714285|
|         Asotin|             2.025|
|         Duplin|2.0000000000000004|
|        Boulder|1.9604709012345676|
+---------------+------------------+
only showing top 20 rows



Q.3) Which states have the best/worst air quality in each year? (Rank states per year
considering pollutants’ level!)


In [87]:
try :
    lines = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./epa_hap_daily_summary-small.csv')
    lines.createOrReplaceTempView('lines')
    t1 = spark.sql("SELECT state_code, collect_set(state_name) AS state_name, sum(arithmetic_mean)/count(arithmetic_mean) AS value, LEFT(date_local, 4) AS year FROM lines GROUP BY state_code, year ORDER BY value DESC")
    t1.show()  
except Exception as e:
    print(e)



+----------+--------------------+------------------+----+
|state_code|          state_name|             value|year|
+----------+--------------------+------------------+----+
|        47|         [Tennessee]|170.40093066666668|1990|
|        80| [Country Of Mexico]|              8.46|1995|
|        26|          [Michigan]| 4.506138716367713|2001|
|        25|     [Massachusetts]| 4.305833285714285|1993|
|        08|          [Colorado]|4.2250000000000005|2017|
|        18|           [Indiana]| 4.098978378378379|1990|
|        17|          [Illinois]| 3.911825163398692|1992|
|        25|     [Massachusetts]|3.4609906122448986|1994|
|        22|         [Louisiana]|3.3643488658536596|1995|
|        44|      [Rhode Island]|         3.3635714|1994|
|        01|           [Alabama]| 3.226314057971014|1996|
|        09|       [Connecticut]|3.0975461538461535|1993|
|        25|     [Massachusetts]|3.0246823529411766|1990|
|        55|         [Wisconsin]| 2.950483333333333|1994|
|        18|  

Q.4) For each state, what is the average distance (in km) of the monitors in that state to
the state center? For simplicity, assume that 1 degree of latitude or logitude equals
to 111 km. (Monitor dispersion per state!)


In [2]:
try :
    lines = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./epa_hap_daily_summary-small.csv')
    lines.createOrReplaceTempView('lines')
    usa = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./usa_states.csv')
    usa.createOrReplaceTempView('states')
    t2=spark.sql("SELECT DISTINCT state_name, address, minlat, latitude, maxlat, minlon, longitude, maxlon, ABS(latitude-(minLat+(maxLat-minLat)/2)) AS dlat, ABS(longitude-(minLon+(maxLon-minLon)/2)) AS dlon FROM lines l, states s WHERE l.state_name=s.name")
    t2.createOrReplaceTempView('t2')
    t3=spark.sql("SELECT state_name, SQRT(POW(AVG(t.dlat)*111, 2)+POW(AVG(t.dlon)*111, 2)) AS hkm FROM t2 t GROUP BY state_name ORDER BY state_name ASC")
    t3.show(10)   

except Exception as e:
    print(e)
#sc.stop()

+-----------+------------------+
| state_name|               hkm|
+-----------+------------------+
|    Alabama|154.49260106651099|
|     Alaska| 578.2547257424047|
|    Arizona|164.10851532123158|
|   Arkansas|139.88499280806286|
| California| 295.9560097738761|
|   Colorado|162.79022014593346|
|Connecticut|48.010830374129405|
|   Delaware|49.430957208141784|
|    Florida| 313.9536451547972|
|    Georgia|171.86252921573762|
+-----------+------------------+
only showing top 10 rows



Q.5) How many sensors there are per quadrant (NW, NE, SE, SW) in each state? To
answer this question, you should approximate each state’s area to a rectangle as
defined in the file “usa_satates.csv”, and divide that area in 4 quadrants (NW,
NE, SE, SW). (Count monitors per sate qudrant!)

In [19]:
try :
    lines = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./epa_hap_daily_summary-small.csv')
    lines.createOrReplaceTempView('lines')
    usa = sqlContext.read.format('com.databricks.spark.csv').options(header=True).load('./usa_states.csv')
    usa.createOrReplaceTempView('usa')
    states= spark.sql("SELECT DISTINCT state_code, state_name, minLat, minLat+ABS((maxLat-minLat)/2) AS centerLat, maxLat, minLon, minLon+((maxLon-minLon)/2) AS centerLon, maxLon, count(DISTINCT latitude+longitude) AS total FROM lines, usa WHERE lines.state_name=usa.name GROUP BY state_code, state_name, minLat, centerLat, maxLat, minLon, CenterLon, maxLon ORDER BY state_code")
    states.createOrReplaceTempView('states')
    #states.show(10)
    ne= spark.sql("SELECT l.state_name, count(DISTINCT latitude+longitude) AS value FROM lines l, states s WHERE l.state_name=s.state_name AND l.latitude>s.centerLat AND l.longitude>s.centerLon GROUP BY l.state_name")
    ne.createOrReplaceTempView('ne')
    #ne.show()
    nw= spark.sql("SELECT l.state_name, count(DISTINCT latitude+longitude) AS value FROM lines l, states s WHERE l.state_name=s.state_name AND l.latitude>s.centerLat AND l.longitude<s.centerLon GROUP BY l.state_name")
    nw.createOrReplaceTempView('nw')
    se= spark.sql("SELECT l.state_name, count(DISTINCT latitude+longitude) AS value FROM lines l, states s WHERE  l.state_name=s.state_name AND l.latitude<s.centerLat AND l.longitude>s.centerLon GROUP BY l.state_name")
    se.createOrReplaceTempView('se')
    sw= spark.sql("SELECT l.state_name, count(DISTINCT latitude+longitude) AS value FROM lines l, states s WHERE  l.state_name=s.state_name AND l.latitude<s.centerLat AND l.longitude<s.centerLon GROUP BY l.state_name")
    sw.createOrReplaceTempView('sw')
    t2= spark.sql("SELECT  l.state_code, l.state_name, nw.value AS NW, ne.value AS NE, sw.value AS SW, se.value AS SE, s.total FROM states s, lines l, usa u, ne, nw, se, sw WHERE l.state_name=ne.state_name AND l.state_name=nw.state_name AND l.state_name=se.state_name AND l.state_name=sw.state_name AND s.state_code=l.state_code GROUP BY l.state_code, l.state_name, nw.value, ne.value, sw.value, se.value, s.total ORDER BY state_name ASC")
    t2.show(10)   
except Exception as e:
    print(e)


+----------+----------+---+---+---+---+-----+
|state_code|state_name| NW| NE| SW| SE|total|
+----------+----------+---+---+---+---+-----+
|        01|   Alabama| 14|  5|  7|  5|   31|
|        02|    Alaska|  3|  4|  3|  2|   12|
|        04|   Arizona| 10|  2| 10| 16|   38|
|        05|  Arkansas|  3|  2|  5|  1|   11|
|        06|California| 84|  2| 16| 68|  170|
|        08|  Colorado| 17| 25|  5|  4|   51|
|        13|   Georgia| 21|  4|  5|  5|   35|
|        17|  Illinois|  2| 32| 14|  1|   49|
|        18|   Indiana| 18| 18|  8|  8|   52|
|        20|    Kansas|  2| 18|  8|  9|   37|
+----------+----------+---+---+---+---+-----+
only showing top 10 rows

